In [ ]:
from keras.models import Sequential
from keras import layers
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D, Flatten, GlobalAveragePooling2D, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint
from keras.models import load_model
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
from string import punctuation
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.applications.resnet import ResNet50 #loading the resnet50 model
from tensorflow.keras import Model

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from textblob import Word 
import csv
import re
import os
from google.colab import drive

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
# mounting google drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# retrieving train and test data
train = '/content/drive/MyDrive/Data/training_set/training_set'
test= '/content/drive/MyDrive/Data/test_set/test_set'

In [ ]:
# normalizing the images
imgNormalization = ImageDataGenerator(preprocessing_function=tf.keras.applications.resnet50.preprocess_input)

# creating train data
train = imgNormalization.flow_from_directory(train, target_size=(32, 32), class_mode='binary', batch_size=32, shuffle=True )
# creating test data
test = imgNormalization.flow_from_directory(test, target_size=(32, 32), class_mode='binary', batch_size=32, shuffle=True)

Found 8005 images belonging to 2 classes.
Found 2023 images belonging to 2 classes.


In [ ]:
# initializing resNet50 model
resNet  = ResNet50( input_shape=(32,32,3), include_top=False)

# freezing restNet layers
for layer in resNet.layers:
  layer.trainable = False
# adding resNet output
l = resNet.output
# adding Maxpool layer
l = GlobalAveragePooling2D()(l)
# adding dense layers
l = Dense(64, activation='relu')(l)
l = Dense(32, activation='relu')(l)

# creating output layer with 1 dense layer with sigmoid activation
output_layer =layers.Dense(1, activation='sigmoid')(l)

# creating early stopping.
es = EarlyStopping(monitor='val_loss', mode='min', patience = 5, restore_best_weights=True)
# creating model checkpoint
mc = ModelCheckpoint("ResNetModel.h5", monitor='val_loss', mode='min', verbose=1, save_best_only=True)
# creating class weights
cw = {0: 1.0, 1: 1.0}

# Creating our model
model = Model(inputs=resNet.input, outputs=output_layer)

# printing summary
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 38, 38, 3)    0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 16, 16, 64)   9472        ['conv1_pad[0][0]']              
                                                                                                  
 conv1_bn (BatchNormalization)  (None, 16, 16, 64)   256         ['conv1_conv[0][0]']             
                                                                                              

In [ ]:
# compile model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy']) 

In [ ]:
# fitting the model
history = model.fit(train, batch_size=10, epochs=7, validation_data=test, callbacks = [es,mc],verbose=1,class_weight=cw)

Epoch 1/7
251/251 [==============================] - ETA: 0s - loss: 0.0000e+00 - accuracy: 0.4999
Epoch 1: val_loss improved from inf to 0.00000, saving model to ResNetModel.h5
251/251 [==============================] - 1444s 6s/step - loss: 0.0000e+00 - accuracy: 0.4999 - val_loss: 0.0000e+00 - val_accuracy: 0.4998
Epoch 2/7
251/251 [==============================] - ETA: 0s - loss: 0.0000e+00 - accuracy: 0.4997
Epoch 2: val_loss did not improve from 0.00000
251/251 [==============================] - 51s 201ms/step - loss: 0.0000e+00 - accuracy: 0.4997 - val_loss: 0.0000e+00 - val_accuracy: 0.4998
Epoch 3/7
251/251 [==============================] - ETA: 0s - loss: 0.0000e+00 - accuracy: 0.4997
Epoch 3: val_loss did not improve from 0.00000
251/251 [==============================] - 54s 216ms/step - loss: 0.0000e+00 - accuracy: 0.4997 - val_loss: 0.0000e+00 - val_accuracy: 0.4998
Epoch 4/7
251/251 [==============================] - ETA: 0s - loss: 0.0000e+00 - accuracy: 0.4997
Epoch 

In [ ]:
# evaluation
result = model.evaluate(test)
result

64/64 [==============================] - 11s 173ms/step - loss: 0.0000e+00 - accuracy: 0.4998


[0.0, 0.49975284934043884]